<a href="https://colab.research.google.com/github/jpsiegel/Projects/blob/master/Tarea1_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tarea 1 Jan Siegel


In [1]:
print("Hello T1")

Hello T1
